# Convolution & pooling
#### it use image processing
#### (Convolution layer -> ReLU) + Pooling layer
#### perceptron process in 1D. so vanish spatial structure. so we need to keep spatial structure and training.
#### convolution steps
#### step1
![step1](https://wikidocs.net/images/page/64066/conv4.png)
#### ...
#### step4
![step4](https://wikidocs.net/images/page/64066/conv7.png)
#### finally
#### feature map
![feature map](https://wikidocs.net/images/page/64066/conv8.png)
#### when convolution, can decide moving range. it is called stride.
#### after convolution, if you want to size up like initial size, can use padding.
#### pooling is very similar convolution. it use for feature max size down and weight numbers down.

In [1]:
# resolve MNIST with CNN(Convolution Neural Network)
# understand model -> (nn.Cov2d + nn.ReLU) + nn.MaxPoold2d
# model architecture -> total 3 layer
# First layer : Convolutional layer
# Convolution(in_channel = 1, out_channel = 32, kernel_size=3, stride=1, padding=1) + ReLU
# maxpool(kernel_size=2, stride=2))
# Second layer : Convolutional layer
# Convolution(in_channel = 32, out_channel = 64, kernel_size=3, stride=1, padding=1) + ReLU
# maxpool(kernel_size=2, stride=2))
# Third layer : Fully-Connected layer
# feature map # batch_size × 7 × 7 × 64 → batch_size × 3136
# fully connected(neurons 10) + Softmax
# import
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [2]:
# check process
# random tensor
inputs = torch.Tensor(1,1,28,28)
print(inputs.shape)
# convolution & pooling
conv1 = nn.Conv2d(1, 32, 3, padding=1)
conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
pool = nn.MaxPool2d(2)
# connect & check shape
out = conv1(inputs)
print(out.shape)
out = pool(out)
print(out.shape)
out = conv2(out)
print(out.shape)
out = pool(out)
print(out.shape)
out = out.view(out.size(0), -1)
print(out.shape)
fc = nn.Linear(3136, 10)
out = fc(out)
print(out.shape)

torch.Size([1, 1, 28, 28])
torch.Size([1, 32, 28, 28])
torch.Size([1, 32, 14, 14])
torch.Size([1, 64, 14, 14])
torch.Size([1, 64, 7, 7])
torch.Size([1, 3136])
torch.Size([1, 10])


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# initial value
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
# hyper parameter
learning_rate = 0.001
training_epochs = 15
batch_size = 100
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True, 
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)
# CNN model class
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # fist layer
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # second layer
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # fully connected layer 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        # fully connected layer weight initialize
        torch.nn.init.xavier_uniform_(self.fc.weight)
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        # fullyconnected layer flatten
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
# definite model
model = CNN().to(device)
# cost function
criterion = torch.nn.CrossEntropyLoss().to(device)
# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# total batch size
total_batch = len(data_loader)
# training
for epoch in range(training_epochs):
    avg_cost = 0
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        avg_cost += cost / total_batch
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.225607827
[Epoch:    2] cost = 0.0629417673
[Epoch:    3] cost = 0.0462888777
[Epoch:    4] cost = 0.0374611467
[Epoch:    5] cost = 0.0313362107
[Epoch:    6] cost = 0.0260468703
[Epoch:    7] cost = 0.0219094232
[Epoch:    8] cost = 0.0185330287
[Epoch:    9] cost = 0.0164066143
[Epoch:   10] cost = 0.0133768078
[Epoch:   11] cost = 0.0100030983
[Epoch:   12] cost = 0.01024774
[Epoch:   13] cost = 0.00819545612
[Epoch:   14] cost = 0.00696233194
[Epoch:   15] cost = 0.00620753365


In [4]:
# dont't training more. so use torch.no_grad()
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

/Users/alex/opt/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/Users/alex/opt/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9887999892234802


In [5]:
# add convolution layer & fully connected layer -> more depth CNN
# device & initialize seed
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
# hyper parameter
learning_rate = 0.001
training_epochs = 15
batch_size = 100
# MNIST DATA
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)
# set dataloader, batch_size
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)
# CNN model class
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L2 ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L3 ImgIn shape=(?, 7, 7, 64)
        #    Conv      ->(?, 7, 7, 128)
        #    Pool      ->(?, 4, 4, 128)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))

        # L4 FC 4x4x128 inputs -> 625 outputs
        self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - self.keep_prob))
        # L5 Final FC 625 inputs -> 10 outputs
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.layer4(out)
        out = self.fc2(out)
        return out
# definte model
model = CNN().to(device)
# cost function included softmax function
criterion = torch.nn.CrossEntropyLoss().to(device)
# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# total batch size
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0
    for X, Y in data_loader: 
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        avg_cost += cost / total_batch
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.193002731
[Epoch:    2] cost = 0.055656828
[Epoch:    3] cost = 0.0382638648
[Epoch:    4] cost = 0.0301043037
[Epoch:    5] cost = 0.0260318089
[Epoch:    6] cost = 0.01972734
[Epoch:    7] cost = 0.017921336
[Epoch:    8] cost = 0.0147547238
[Epoch:    9] cost = 0.0145895211
[Epoch:   10] cost = 0.0115573956
[Epoch:   11] cost = 0.0111309709
[Epoch:   12] cost = 0.00801258907
[Epoch:   13] cost = 0.00944773387
[Epoch:   14] cost = 0.00925426371
[Epoch:   15] cost = 0.00682048406


In [6]:
# no more training, so use torch.no_grad()
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())
# result => more layer give but no more accuracy. so need efficient layer.

Accuracy: 0.987500011920929
